# Versão final do modelo do TCC

Lucca Cerf Costa | **RA00304770**   
Letícia Lopes Bueno | **RA00297719**

---

## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import folium
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

import requests
import warnings
from folium.plugins import HeatMap
import json

from shapely.geometry import shape

---
## Dados das Cidades

In [ ]:
# Lendo o documento que traz diversas informações sobre população, IDH, PIB per capita etc.
dfc = pd.read_csv(r'DadosCidades/BRAZIL_CITIES_rev2022.csv')

# Lista das colunas para manter
manter = ['CITY','STATE', 'IBGE_RES_POP','IBGE_DU','IDHM','GVA_TOTAL','GDP','Cars','Motorcycles','POST_OFFICES']

# Removendo todas as colunas que não estão na lista de colunas para manter
remover = [coluna for coluna in dfc.columns if coluna not in manter]
dfc = dfc.drop(columns=remover)
dfc.head()

# Precisamos transformar o IDH de cada municipio no IDH de cada estado
# Faremos essa transformação por agregação ponderada
# Primeiro, calculamos o produto entre a população residente (IBGE_RES_POP) e o IDHM de cada município
dfc['Pop_IDHM'] = dfc['IBGE_RES_POP'] * dfc['IDHM']

# Em seguida, agrupamos os dados por estado e calculamos a soma da população,a soma dos domicílios e 
# a soma dos produtos população-IDHM
dfc2 = dfc.groupby('STATE').agg({
    'IBGE_RES_POP': 'sum',
    'IBGE_DU': 'sum',
    'Pop_IDHM': 'sum',
    'GVA_TOTAL': 'sum',
    'GDP': 'sum',
    'Cars': 'sum',
    'Motorcycles': 'sum',
    'POST_OFFICES': 'sum'
}).reset_index()

# Calculamos o IDH ponderado dividindo a soma dos produtos população-IDHM pela soma da população
dfc2['IDH_Ponderado'] = dfc2['Pop_IDHM'] / dfc2['IBGE_RES_POP']

# Descartamos a coluna do IDH-M
dfc2.drop(columns=['Pop_IDHM'], inplace=True)

# Renomeamos as colunas conforme necessário
dfc2 = dfc2.rename(columns={
    'STATE':'Estado',
    'IBGE_RES_POP': 'População',
    'IBGE_DU': 'Domicilios',
    'GVA_TOTAL': 'GVA',
    'GDP': 'PIB',
    'Cars': 'Carros',
    'Motorcycles': 'Motos',
    'POST_OFFICES': 'Correios'
})

dfc2.head()

---
## Dados de Alimentação e Produção

In [ ]:
# Esse dataset traz a Segurança ALimentar de cada município e a quantidade de empresas de cada ramo de produção/alimentação
dfs = pd.read_excel(r'Dados/DadosBR.xlsx').rename(columns={'Região':'Estado'})

# Esses dados de Quantidade Produzida estão errados. Dados mais preciso estão no próximo dataframe
dfs.drop(columns=['Quantidade Produzida (t)'], inplace=True)
dfs.head()

In [ ]:
# Esse dataset traz o número de animais em cada estado e o faturamento da agricultura em milhares de reais.
dfp = pd.read_excel('Dados/ProdAlimentar.xlsx')
dfp.head()

---
## Dados de Rodovias

In [ ]:
# Esse dataset traz a porcentagem de estradas pavimentadas em cada estado
dfr = pd.read_excel('DadosRodovias/df3_salvo.xlsx')

# Calculando a porcentagem de pavimentação
dfr['% pavimentada'] = ((dfr['pavimentada'] / (dfr['pavimentada'] + dfr['não-pavimentada'])) * 100).round(2)
dfr.drop(columns=['pavimentada','não-pavimentada'], inplace=True)

# Renomeando colunas para depois juntar os datasets baseando-se nos estados
dfr.rename(columns={'Estados':'Estado'}, inplace=True)
dfr.head()

---
## Dados EPSAN2018

In [ ]:
''' Montando um dataset baseado em diversas informações sobre alimentação. Todos os dados dos 4 datasets abaixo
são oficiais do Mapeamento de Segurança Alimentar e Nutricional (MapaSAN). Outros estabelecimentos semelhantes podem
existir mas não foram registrados até o momento de coleta dos dados (2018)'''

# Banco de Alimentos
dfBA = pd.read_csv("DadosEPSAN2018/Banco_Alimentos.csv", encoding='latin1', delimiter = ";")
cont1 = dfBA['v9170'].value_counts().sort_index()

# Cozinha Comunitária
dfCC = pd.read_csv("DadosEPSAN2018/Cozinha_Comunitaria.csv", encoding='latin1', delimiter = ";")
cont2 = dfCC['v9141'].value_counts().sort_index()

# Restaurante Popular
dfRP = pd.read_csv("DadosEPSAN2018/Restaurante_Popular.csv", encoding='latin1', delimiter = ";")
cont3 = dfRP['v9109'].value_counts().sort_index()

# Feira Livre
dfFL = pd.read_csv("DadosEPSAN2018/Feira_Livre.csv", encoding='latin1', delimiter = ";")
cont4 = dfFL['v9966'].value_counts().sort_index()

In [ ]:
# Lista com todos os estados do Brasil + DF
estados = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
           'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
           'RS', 'SC', 'SE', 'SP', 'TO']

# Função para garantir que todos os estados estejam presentes
def garantir_todos_estados(series, estados):
    todos_estados_series = pd.Series(0, index=estados)
    todos_estados_series.update(series)
    return todos_estados_series

In [ ]:
# Garantindo que todas as variáveis tenham todos os estados
ba = garantir_todos_estados(cont1, estados)
cc = garantir_todos_estados(cont2, estados)
rp = garantir_todos_estados(cont3, estados)
fl = garantir_todos_estados(cont4, estados)

dfe = pd.DataFrame({
    'Estado': estados,
    'Banco de Alimentos': ba,
    'Cozinha Comunitária': cc,
    'Restaurante Popular': rp,
    'Feira Livre': fl
}).reset_index(drop=True)

dfe.head(5)

### Dados dos hospitais e postos de saúde

In [ ]:
dfh = pd.read_csv(r'DadosInfra/Hospital.csv')

# Removendo colunas
dfh.drop(dfh.columns[np.r_[2:17, 18:20, 22:31,32:37]], axis=1, inplace=True)

# Renomeando colunas
novos_nomes = {
    'address': 'endereço',
    'categories/0': 'categoria',
    'city': 'cidade',
    'location/lat': 'latitude',
    'location/lng': 'longitude',
    'state': 'estado'
}

dfh.rename(columns=novos_nomes, inplace=True)

# Removendo valores não-relacionados
valores_a_remover = ['Hospital equipment and supplies',
                     'Veterinarian','Public university','Health insurance agency',
                     'Association or organization','Blueprint service',
                     'Transit station','Apartment building','Resort hotel']

dfh = dfh[~dfh['categoria'].isin(valores_a_remover)]

# Removendo NaN
dfh = dfh.dropna(subset=['categoria'])
dfh.head(5)

In [ ]:
dfps = pd.read_csv(r'DadosInfra/Posto de Saúde.csv')

# Removendo colunas
dfps.drop(dfps.columns[np.r_[1:11, 13:15, 17:26, 27:32]], axis=1, inplace=True)

# Renomeando colunas
novos_nomes = {
    'address': 'endereço',
    'categoryName': 'categoria',
    'city': 'cidade',
    'location/lat': 'latitude',
    'location/lng': 'longitude',
    'state' : 'estado'
}

dfps.rename(columns=novos_nomes, inplace=True)

# Removendo valores não-relacionados
valores_a_remover = ['Public educational institution','Compressed natural gas station','Garden','Perfume store',
                     'Electric vehicle charging station','Condominium complex','Gated community',
                     'Municipal office education','Health insurance agency','Lodging','Transportation service',
                     'Museum','Shoe store','Public works department','State Dept of Culture','Telecommunications engineer',
                     'Supermarket','Bus station','Park','State government office', 'Judaica store','Primary school',
                     'Tailor','Wine store', 'Apartment building','Occupational medical physician',
                     'Pharmaceutical products wholesaler', 'Housing complex', 'Public bathroom',
                     'County government office', 'Junkyard','Housing development', 'Driving test center',
                     'Local government office', 'Physician referral service', 'City district office','Pilates studio',
                     'Non-profit organization','Education center', 'Dressmaker', 'School','Government office',
                     'Public health department','Gas station','Government','Bus stop','Municipal Social Development',
                     'Store','Pharmaceutical company','Parking lot','Nurse practitioner','Health and beauty shop',
                     'College','Villa', 'City Hall','Elementary school','Post office','Association or organization',
                     'Alternative medicine practitioner', 'City government office','State police', 'Cultural association',
                     'Veterinary care', 'General practitioner','Utility contractor', 'ATM','Nursing school']

dfps = dfps[~dfps['categoria'].isin(valores_a_remover)]

# Removendo NaN
dfps = dfps.dropna(subset=['categoria'])
dfps.head(5)

In [ ]:
# Juntando os dataframes de Hospitais e Postos de Saúde
dfhps = pd.merge(dfh, dfps, how='outer')

# Removendo linhas duplicadas com base nas colunas "latitude" e "longitude"
dfhps = dfhps.drop_duplicates(subset=['latitude', 'longitude']).sort_values(by='cidade').dropna(subset=['latitude'])

# Removendo linhas com valores nulos na coluna 'estado'
dfhps = dfhps.dropna(subset=['estado'])

# Função para substituir os nomes completos pelos códigos de estado
def substituir_estado(nome_completo):
    mapeamento_estados = {
        'State of Acre': 'AC', 'State of Alagoas': 'AL', 'State of Amapá': 'AP',
        'State of Amazonas': 'AM', 'State of Bahia': 'BA', 'Ceará': 'CE',
        'Federal District': 'DF', 'State of Espírito Santo': 'ES', 'State of Goiás': 'GO',
        'State of Maranhão': 'MA', 'State of Mato Grosso': 'MT', 'State of Mato Grosso do Sul': 'MS',
        'State of Minas Gerais': 'MG', 'State of Pará': 'PA', 'State of Paraíba': 'PB',
        'State of Paraná': 'PR', 'State of Pernambuco': 'PE', 'State of Piauí': 'PI',
        'State of Rio de Janeiro': 'RJ', 'State of Rio Grande do Norte': 'RN',
        'State of Rio Grande do Sul': 'RS', 'State of Rondônia': 'RO', 'State of Roraima': 'RR',
        'State of Santa Catarina': 'SC', 'State of São Paulo': 'SP', 'State of Sergipe': 'SE',
        'State of Tocantins': 'TO', 'Rondônia': 'RO', 'Pernambuco': 'PE',
        'Santa Catarina': 'SC', 'Minas Gerais': 'MG', 'Goiás': 'GO'
    }
    if nome_completo in mapeamento_estados:
        return mapeamento_estados[nome_completo]
    elif isinstance(nome_completo, str):
        for estado in mapeamento_estados:
            if estado in nome_completo:
                return mapeamento_estados[estado]
    return nome_completo

# Aplicando a função de substituição à coluna 'estado'
dfhps['estado'] = dfhps['estado'].apply(substituir_estado)

# Resetando o index
dfhps.reset_index(drop=True, inplace=True)

dfhps.head(5)

In [ ]:
dfhps.rename(columns={'estado': 'Estado'}, inplace=True)

# Trocando "Federal District" por "DF" na coluna 'Estado'
dfhps['Estado'] = dfhps['Estado'].replace('Federal District', 'DF')

# Agrupando por estado
dfhps2 = dfhps.groupby('Estado').size()

# Convertendo o resultado para um DataFrame para melhor visualização
dfhps2 = dfhps2.reset_index(name='Hospitais e Postos de Saúde')
dfhps2.head(5)

---
## Dados Vigitel
### Dados sobre obesidade, alimentação, uso de eletrônicos e fumo

In [ ]:
dv = pd.read_excel('DadosObesidade/DadosVigitel.xlsx')

# Separando dados não normalizados para o mapa
dv_mapa = pd.read_excel('DadosObesidade/DadosVigitel.xlsx')

# Separando os dados de obesidade
peso = dv[['Estado','% Sobrepeso', '% Obesidade']]

# Removendo a porcentagem de obesidade e sobrepeso do modelo (serão o target)
dv.drop(columns=['% Obesidade','% Sobrepeso'], inplace = True)
dv.head(5)

In [ ]:
peso.head(5)

---
### Juntando os datasets

In [ ]:
# Criando a lista de DataFrames
df_list = [dfc2, dfs, dfe, dfhps2, dfr, dfp, dv]

# Mesclando os DataFrames
df_final = df_list[0]  # Começa com o primeiro DataFrame da lista
for df in df_list[1:]:
    df_final = pd.merge(df_final, df, on='Estado', how='outer')
    
df_final['Pecuária(cabeças)']=df_final['Pecuária(cabeças)'].apply(lambda x: '{:.0f}'.format(x))
df_final.head(5)

---
### Tratamento dos dados

In [ ]:
df = df_final

# Ordenando os estados pela pontuação e colocando os dados sobre obesidade
df = pd.merge(df, peso[['Estado',
                        '% Sobrepeso',
                        '% Obesidade']], on='Estado', how='right').sort_values(by='Estado', ascending=False)

# Normalizando os dados (exceto Estado)
scaler = MinMaxScaler()
df[df.columns[1:]] = scaler.fit_transform(df[df.columns[1:]])

df.head(5)

In [ ]:
dfcorr = df.drop(columns=['Estado'])

# Calculando a correlação com '% Sobrepeso' e '% Obesidade'
correlation_sobrepeso = dfcorr.corrwith(dfcorr['% Sobrepeso'])
correlation_obesidade = dfcorr.corrwith(dfcorr['% Obesidade'])

# Exibindo as correlações
print("Correlação com '% Sobrepeso':")
print(correlation_sobrepeso)
print("\nCorrelação com '% Obesidade':")
print(correlation_obesidade)

In [ ]:
import matplotlib.pyplot as plt

# Excluir as variáveis % Sobrepeso e % Obesidade
correlation_without_obesity = correlation_sobrepeso.drop(['% Sobrepeso', '% Obesidade'])

# Separar as variáveis em grupos com base na correlação
positivas = correlation_without_obesity[correlation_without_obesity > 0].nlargest(5)
neutras = correlation_without_obesity[(correlation_without_obesity >= -0.1) & (correlation_without_obesity <= 0.1)].nlargest(3)
negativas = correlation_without_obesity[correlation_without_obesity < 0].nsmallest(5)

# Juntar os grupos
variaveis_selecionadas = pd.concat([positivas, neutras, negativas])

# Criar o gráfico de barras
plt.figure(figsize=(10, 6))
variaveis_selecionadas.plot(kind='bar', color=['green' if x > 0 else 'red' for x in variaveis_selecionadas])
plt.title('Correlação com % Sobrepeso')
plt.xlabel('Variável')
plt.ylabel('Correlação')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


---
### Modelo

In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# Definindo a coluna de target
target = '% Obesidade'

# Removendo a coluna 'Estado' para treinamento
train_data = df.drop(columns=['Estado'])

# Configurando o AutoGluon
predictor = TabularPredictor(label=target, problem_type='regression').fit(train_data)

# Importância das Features
feature_importance = predictor.feature_importance(train_data)

# Armazenando a importância das features em um DataFrame
feature_importance_df = pd.DataFrame(feature_importance)

# Avaliando o modelo
performance = predictor.evaluate(train_data)

# Armazenando as métricas de desempenho em um DataFrame
performance_df = pd.DataFrame(performance, index=[0])

# Obtendo o melhor modelo
best_model = predictor.get_model_best()
print(f"Melhor modelo: {best_model}")

# Fazendo previsões no conjunto de treino
predictions = predictor.predict(train_data.drop(columns=[target]))

# Armazenando as previsões em um DataFrame
predictions_df = train_data.copy()
predictions_df[target + '_predicted'] = predictions

# Avaliando novamente o modelo para verificar o desempenho
performance_after_predictions = predictor.evaluate(train_data)
performance_after_predictions_df = pd.DataFrame(performance_after_predictions, index=[0])

In [ ]:
feature_importance_df

In [ ]:
performance_df

In [ ]:
predictions_df.head(5)

In [ ]:
from sklearn.metrics import r2_score

# y_true são os valores reais, y_pred são as previsões do modelo
y_true = predictions_df['% Obesidade']
y_pred = predictions_df['% Obesidade_predicted']

# Calcular R²
r2 = r2_score(y_true, y_pred)

# Converter R² para porcentagem
accuracy = r2 * 100

print(f"A precisão do modelo é de {accuracy:.2f}%")

---
## Mapas
### Mapa dos danos em rodovias

In [ ]:
# Carregando o arquivo GeoJSON
with open('brazil-states.geojson') as f:
    geojson_data = json.load(f)

df_1 = pd.read_csv(r'C:\Users\lucca\TCC\DadosRodovias\icm_08_2022.csv', delimiter=';')

# Corrigindo o formato das coordenadas (substituir vírgulas por pontos)
df_1['Longitude'] = df_1['Longitude'].str.replace(',', '.').astype(float)
df_1['Latitude'] = df_1['Latitude'].str.replace(',', '.').astype(float)

# Criando o mapa
latitude_media = df_1['Latitude'].mean()
longitude_media = df_1['Longitude'].mean()
mapa = folium.Map(location=[latitude_media, longitude_media], zoom_start=5)

# Adicionando marcadores simples para cada ponto
for _, row in df_1.iterrows():
    folium.Circle(
        location=[row['Latitude'], row['Longitude']],
        radius=0.2,  # Tamanho do marcador
        color='red',  # Cor do marcador
        fill=True,
        fill_color='red',
        fill_opacity=0.3
    ).add_to(mapa)

# Adicionando o GeoJSON ao mapa
folium.GeoJson(geojson_data).add_to(mapa)

# Salvando o mapa como um arquivo HTML
# mapa.save("mapa_icm_12_2022.html")

mapa

---
### Mapa das estruturas de saúde

In [ ]:
# Criando o mapa
mapa = folium.Map(location=[-15.788, -47.879], zoom_start=4)

# Convertendo as coordenadas (latitude e longitude) em uma lista de tuplas
dados_calor = dfhps[['latitude', 'longitude']].values.tolist()

# Criando o mapa de calor
heat_map = HeatMap(dados_calor, radius=12, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'})
mapa.add_child(heat_map)

mapa

# Salvar o mapa como um arquivo HTML
#mapa.save("mapa.html")

In [ ]:
# Criar o mapa
mapa = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

# Adicionando choropleth
folium.Choropleth(
    geo_data=geojson_data,
    data=dv_mapa,
    columns=['Estado', '% Hipertensão'],
    key_on='feature.properties.sigla',  # Usar a sigla do estado para correspondência
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='% Hipertensão'
).add_to(mapa)

# Função para obter o centro de um polígono
def get_centroid(geojson_feature):
    polygon = shape(geojson_feature['geometry'])
    centroid = polygon.centroid
    return centroid.y, centroid.x

# Adicionando rótulos permanentes com base no centro dos estados
for feature in geojson_data['features']:
    coords = get_centroid(feature)
    estado = feature['properties']['sigla']
    folium.Marker(
        location=coords,
        icon=folium.DivIcon(html=f'<div style="font-size: 8pt; color: black;">{estado}</div>')
    ).add_to(mapa)

# Salvar o mapa como um arquivo HTML
#mapa.save("mapa_hipertensao.html")

mapa

In [ ]:
with open('brazil-states.geojson') as f:
    geojson_data = json.load(f)

mapa = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

folium.Choropleth(
    geo_data=geojson_data,
    data=dv_mapa,
    columns=['Estado', '% Diabetes'],
    key_on='feature.properties.sigla',  # Usar a sigla do estado para correspondência
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='% Diabetes'
).add_to(mapa)

def get_centroid(geojson_feature):
    polygon = shape(geojson_feature['geometry'])
    centroid = polygon.centroid
    return centroid.y, centroid.x

for feature in geojson_data['features']:
    coords = get_centroid(feature)
    estado = feature['properties']['sigla']
    folium.Marker(
        location=coords,
        icon=folium.DivIcon(html=f'<div style="font-size: 8pt; color: black;">{estado}</div>')
    ).add_to(mapa)

# Salvar o mapa como um arquivo HTML
#mapa.save("mapa_diabetes.html")

mapa

---
## Referências Bibliográficas

https://aplicacoes.mds.gov.br/sagi/mapasan/index.php?pasta=&el=%2F2018%2FBASE+DE+DADOS+E+RELAT%C3%93RIOS%2FNacional%2F4.+Epsans

https://cidades.ibge.gov.br/

https://www.kaggle.com/datasets/crisparada/brazilian-cities?resource=download

https://www.mds.gov.br/webarquivos/cidadania/Caisan/MapaSan/MAPASAN%202018.pdf

https://www.mds.gov.br/webarquivos/publicacao/seguranca_alimentar/mapa_san_resultados_preliminares.pdf

https://dados.gov.br/home

https://dados.gov.br/dados/conjuntos-dados/equipamentos-publicos-de-seguranca-alimentar-e-nutricional---2019

https://dados.gov.br/dados/conjuntos-dados/condicoes-do-pavimento1

https://pesquisarodovias.cnt.org.br/login?returnUrl=%2Fmapa

https://apify.com/compass/crawler-google-places

https://abeso.org.br/wp-content/uploads/2021/07/vigitel_brasil_2019_vigilancia_fatores_risco-1-2.pdf